In [1]:
# First we import the necessary libraries
import sys
import pygcdl
import geopandas as gpd
import os
import pandas as pd
import numpy as np
from pathlib import Path

# Create the pygcdl object
pygcdl_obj = pygcdl.PyGeoCDL()

In [4]:
print(pygcdl_obj.list_datasets())

{'DaymetV4': 'Daymet Version 4', 'GTOPO30': 'Global 30 Arc-Second Elevation', 'MODIS_NDVI': 'MODIS NDVI Data, Smoothed and Gap-filled, for the Conterminous US: 2000-2015', 'NASS_CDL': 'NASS Cropland Data Layer', 'NLCD': 'National Land Cover Database', 'PRISM': 'PRISM', 'RAPV3': 'Rangeland Analysis Platform Version 3', 'SMAP-HB1km': 'SMAP HydroBlocks - 1 km', 'Soilgrids250mV2': 'SoilGrids — global gridded soil information', 'VIP': 'Vegetation Index and Phenology (VIP) Vegetation Indices Daily Global 0.05Deg CMG V004'}


In [5]:
print(pygcdl_obj.get_dataset_info("PRISM"))

{'name': 'PRISM', 'id': 'PRISM', 'url': 'https://prism.oregonstate.edu/', 'description': 'The PRISM Climate Group gathers climate observations from a wide range of monitoring networks, applies sophisticated quality control measures, and develops spatial climate datasets to reveal short- and long-term climate patterns. The resulting datasets incorporate a variety of modeling techniques and are available at multiple spatial/temporal resolutions, covering the period from 1895 to the present.', 'provider_name': 'PRISM Climate Group, Oregon State University', 'provider_url': 'https://prism.oregonstate.edu/', 'grid_size': 0.041666666666666664, 'grid_unit': 'degrees', 'vars': {'ppt': 'total precipitation (rain+melted snow)', 'tmean': 'mean temperature (mean of tmin and tmax)', 'tmin': 'minimum temperature', 'tmax': 'maximum temperature', 'tdmean': 'mean dew point temperature', 'vpdmin': 'minimum vapor pressure deficit', 'vpdmax': 'maximum vapor pressure deficit'}, 'temporal_resolution': {'yea

We can upload a geometry as:
- A geojson file
- A shapefile
- A zipfile containing shapefile files
- A csv file (point data only?)
- A geopandas dataframe

The GCDL can only handle generate polygon subsets of single polygons, or multipolygon objects that contain only one polygon. If the user attempts to upload a geopandas dataframe that contains multiple polygons, then pygcdl calculates the ratio between the area of the union of polygons, and the area of the convex hull. If the union of polygons covers at least 80% of the area of the convex hull, then the pygcdl uploads the convex hull. Otherwise, pygcdl uploads each polygon individually, and returns a list of GUIDs.

If the user uploads a file, the file contents are not checked. So, it is possible for a user to upload a multipolygon file without any errors or warnings, and then run into errors when trying to use the GUID for that upload to download a polygon subset.

In [2]:
# Upload a geojson
guid0 = pygcdl_obj.upload_geometry("sample_data/clay_aoi.geojson")
print(guid0)

5e4eb22d-af3d-4c14-bb39-b4c31daa8a0c


In [6]:
# Upload a shapefile, creates a zip file
guid1 = pygcdl_obj.upload_geometry("sample_data/jer_bounds_sf.shp")
print(guid1)

0ecc372f-2485-4985-8785-7f3df84c1eba


In [9]:
# Upload a zipfile, assumes contents are .shp + associated files
guid2 = pygcdl_obj.upload_geometry("sample_data/four_points.zip")
print(guid2)

c0cc78bc-b407-4212-931e-9e2ba66fa242


In [10]:
# Upload a CSV of points
#guid3 = pygcdl_obj.upload_geometry("sample_data/...csv")

In [3]:
# CAUSES ERROR: upload a .shp without associated files in same directory
#guid4 = pygcdl_obj.upload_geometry("sample_data/lonely_shapefile.shp")

In [3]:
dsvars = pd.DataFrame(
    [["PRISM", "ppt"], ["MODIS_NDVI", "NDVI"]], 
    columns=["dataset", "variable"])
print(dsvars)

      dataset variable
0       PRISM      ppt
1  MODIS_NDVI     NDVI


In [4]:
years = "2008"
months = "7:8"
grain_method = "any"

In [5]:
spat_res = 1000 # in units of meters
resample_method = "bilinear"

In [7]:
output_path = Path("output_test")
if not output_path.is_dir():
    output_path.mkdir()

In [10]:
output_files = pygcdl_obj.download_polygon_subset(
    dsvars=dsvars, 
    years=years,
    months=months,
    grain_method=grain_method,
    t_geom=guid1,
    dsn=output_path,
)

http://127.0.0.1:8000/subset_polygon?datasets=PRISM%3Appt%3BMODIS_NDVI%3ANDVI&geom_guid=0ecc372f-2485-4985-8785-7f3df84c1eba&resample_method=nearest&years=2008&months=7%3A8&grain_method=any&validate_method=strict&output_format=geotiff
Written file:  output_test\gcdl_subset_240426_140936_1.zip


NameError: name 'ZipFile' is not defined